In [ ]:
import pathlib

model_data_path = pathlib.Path("./model_data/").resolve()
db_path = pathlib.Path("./sample_data/staffing.db").resolve()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(f"sqlite:///{db_path}")

In [ ]:
query_assignments = """
SELECT
  name, start_time, end_time
FROM assignments
INNER JOIN staff on staff.id == assignments.staff_id
INNER JOIN shifts on shifts.id == assignments.shift_id
"""

query_uncovered = """
SELECT
  id, start_time, end_time, IfNull(assigned, 0) as assigned, staff_count as required, qualification_id
FROM shifts
LEFT JOIN (
    SELECT
    shift_id, count(staff_id) as assigned
    FROM assignments
    GROUP BY shift_id
) on shifts.id == shift_id
WHERE start_time BETWEEN '2023-04-01T00:00:00Z' and '2023-05-30T00:00:00Z'
"""

with engine.connect() as connection:
    assignments = pd.read_feather(model_data_path / "assignments.feather")
    assignments.to_sql("assignments", connection, if_exists="replace", index_label="id")
    assignments = pd.read_sql(query_assignments, connection, parse_dates=["start_time", "end_time"])
    uncovered = pd.read_sql(query_uncovered, connection, parse_dates=["start_time", "end_time"]).query("assigned < required")

In [ ]:
assignments

In [ ]:
uncovered